In [1]:
import logging
import time

import gensim
import nltk.stem
import spacy

from ifai import *

# logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# loading model
model = load_model('models/GoogleNews-vectors-negative300.bin')

2018-03-27 14:02:17,833 : INFO : loading EuclideanKeyedVectors object from models/GoogleNews-vectors-negative300.cache


[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:777)>


2018-03-27 14:02:31,716 : INFO : loading syn0norm from models/GoogleNews-vectors-negative300.cache.syn0norm.npy with mmap=None
2018-03-27 14:02:41,422 : INFO : loading syn0 from models/GoogleNews-vectors-negative300.cache.syn0.npy with mmap=None
2018-03-27 14:02:56,635 : INFO : loaded models/GoogleNews-vectors-negative300.cache


In [8]:
descriptions = open('descriptions.txt')
texts = [line for line in descriptions.readlines()[:]]
i = 0

for text in texts:
    i += 1
    print()
    print(i, ",", text)
    print("possible actions:", possible_actions(model, text))


1 , You are standing at the end of a road before a small brick building. Around you is a forest.  A small stream flows out of the building and down a gully.

possible actions: ['build building', 'bring building', 'go building', 'develop building', 'convert building', 'building building', 'weather building', 'shelter building', 'fall building', 'providing building', 'housing building', 'protecting building', 'cast building', 'face building', 'be building']

2 , At your feet all the water of the stream splashes into a 2-inch slit in the rock. Downstream the streambed is bare rock.

possible actions: []

3 , You are in a 20-foot depression floored with bare dirt. Set into the dirt is a strong steel grate mounted in concrete. A dry streambed leads into the depression. The grate is locked.

possible actions: []

4 , It is a small bedroom with a faded carpet and old wallpaper. There is a washbasin, a chair with a tatty dressing gown slung over it, and a window with the curtains drawn. Near 

In [15]:
i = 0
for text in texts:
    i += 1
    print(i, ",", text)
    nouns = get_noun_from_text(text)
    
    for noun in nouns: 
        adj = get_adjectives_for_noun(model, noun)
        verb = get_verbs_for_noun(model, noun)
        print(noun)
        print("adj:", adj)
        print("verb:", verb)
    print()

1 , You are standing at the end of a road before a small brick building. Around you is a forest.  A small stream flows out of the building and down a gully.

building
adj: ['bright', 'strong', 'drab', 'dark', None]
verb: ['build', 'bring', 'go', 'develop', 'convert', 'building', 'weather', 'shelter', 'fall', 'providing', 'housing', 'protecting', 'cast', 'face', 'be']
building
adj: ['bright', 'strong', 'drab', 'dark', None]
verb: ['build', 'bring', 'go', 'develop', 'convert', 'building', 'weather', 'shelter', 'fall', 'providing', 'housing', 'protecting', 'cast', 'face', 'be']

2 , At your feet all the water of the stream splashes into a 2-inch slit in the rock. Downstream the streambed is bare rock.


3 , You are in a 20-foot depression floored with bare dirt. Set into the dirt is a strong steel grate mounted in concrete. A dry streambed leads into the depression. The grate is locked.


4 , It is a small bedroom with a faded carpet and old wallpaper. There is a washbasin, a chair with a

In [5]:
def spacy_noun(text):
# """extract noun from given text"""

    # tokenize the given text with SpaCy
    nlp = spacy.load('en')
    doc = nlp(text)

    # collect lemmatized nouns from tokens
    wnl = nltk.stem.WordNetLemmatizer()
    nouns = [wnl.lemmatize(chunk.root.text).lower() for chunk in doc.noun_chunks]
    return nouns

def w2v_mani_test(model, nouns, big, small):
    """rank inputs nouns from most manipulative to least manipulative"""
    # anchor x_axis by using forest & tree vector difference
    x_axis = model.word_vec(big) - model.word_vec(small)
    dic = {}

    # map the noun's vectors to the x_axis and spit out a list from small to big
    for noun in nouns:
        if noun not in dic:
            vec = model.word_vec(noun)
            dic[noun] = np.dot(vec, x_axis)
    sorted_list = sorted(dic.items(), key=(lambda kv: kv[1]))
    return sorted_list


descriptions = open('descriptions.txt')
texts = [line for line in descriptions.readlines()[:]]
i = 0

for text in texts:
    i += 1
    nouns = spacy_noun(text)
    print(i, ",", text)
    print("spacy:", spacy_noun(text))
    print("w2v:", w2v_rank_manipulability(model, nouns))
    print("w2v u_c:", w2v_mani_test(model, nouns, "university", "college"))
    print("w2v brain_neuron", w2v_mani_test(model, nouns, "brain", "neuron"))
    print("w2v tree_branch", w2v_mani_test(model, nouns, "tree", "branch"))
    print("w2v city_town", w2v_mani_test(model, nouns, "city", "town"))
    print("w2v big_small", w2v_mani_test(model, nouns, "big", "small"))
    print("w2v atom_protom", w2v_mani_test(model, nouns, "atom", "proton"))
#     print("umbel:",get_noun_from_text(text))
    print()

1 , You are standing at the end of a road before a small brick building. Around you is a forest.  A small stream flows out of the building and down a gully.

spacy: ['you', 'end', 'road', 'building', 'you', 'forest', 'stream', 'building']
w2v: [('you', -0.68128884), ('building', -0.33525023), ('end', 0.00995297), ('road', 0.13379501), ('stream', 0.18704017), ('forest', 6.0384345)]
w2v u_c: [('you', -0.7581904), ('stream', -0.23049074), ('road', -0.1732103), ('end', 0.10318915), ('forest', 0.60390973), ('building', 0.6487004)]
w2v brain_neuron [('forest', -0.9519286), ('building', -0.501072), ('stream', -0.48052895), ('you', -0.08243513), ('road', 0.24495238), ('end', 0.31892067)]
w2v tree_branch [('building', -0.16097158), ('end', 0.15285613), ('stream', 0.3686401), ('you', 0.9069171), ('road', 1.2082683), ('forest', 2.718333)]
w2v city_town [('forest', -0.76510996), ('road', -0.52941084), ('stream', -0.41183645), ('you', -0.042304583), ('end', -0.0022125095), ('building', 0.41160822)]

8 , You are walking down a path through a forest. An endless orchestra of whispering trees and chirping birds play around you. The sun had already dipped below the mountains and the night rolls over the sky.

spacy: ['you', 'path', 'forest', 'orchestra', 'tree', 'bird', 'you', 'sun', 'mountain', 'night', 'sky']
w2v: [('tree', -5.7976556), ('sun', -0.843465), ('you', -0.68128884), ('sky', -0.67014205), ('bird', -0.5377194), ('night', -0.2771899), ('path', -0.22209328), ('orchestra', -0.096692905), ('mountain', 1.0250318), ('forest', 6.0384345)]
w2v u_c: [('you', -0.7581904), ('path', -0.36832374), ('night', -0.2258786), ('sky', -0.1915319), ('bird', -0.0771858), ('mountain', -0.026418194), ('sun', 0.09703329), ('tree', 0.109052494), ('forest', 0.60390973), ('orchestra', 0.75147104)]
w2v brain_neuron [('path', -1.234421), ('bird', -1.200436), ('forest', -0.9519286), ('sky', -0.87818104), ('tree', -0.651603), ('sun', -0.28784359), ('you', -0.08243513), ('night', -0.037605464), ('orchestra

13 , You continue walking along the side of the lake. The water is calm. A chorus of crickets chime in with the beats of your footsteps. An owl hoots from somewhere in the dark trees.

spacy: ['you', 'side', 'lake', 'water', 'chorus', 'beat', 'footstep', 'owl', 'tree']
w2v: [('tree', -5.7976556), ('side', -0.7305247), ('you', -0.68128884), ('footstep', -0.22296512), ('owl', 0.07071954), ('chorus', 0.08172928), ('beat', 0.62687355), ('water', 0.6583246), ('lake', 0.82478356)]
w2v u_c: [('you', -0.7581904), ('beat', -0.45078325), ('chorus', -0.069244295), ('water', 0.059438393), ('tree', 0.109052494), ('footstep', 0.15314732), ('lake', 0.24444532), ('owl', 0.2903127), ('side', 0.33414254)]
w2v brain_neuron [('footstep', -1.2511325), ('chorus', -0.9938238), ('owl', -0.8128423), ('tree', -0.651603), ('you', -0.08243513), ('beat', -0.061882257), ('lake', 0.08830705), ('water', 0.2309425), ('side', 0.49420834)]
w2v tree_branch [('beat', -0.5199853), ('side', 0.36119425), ('chorus', 0.4778606

20 , You are at the top of the Great Canyon on its south wall.  From here there is a marvelous view of the Canyon and parts of the Frigid River upstream.  Across the canyon, the walls of the White Cliffs still appear to loom far above.  Following the Canyon upstream (north and northwest), Aragain Falls may be seen, complete with rainbow. Fortunately, my vision is better than average and I can discern the top of the Flood Control Dam #3 far to the distant north.  To the west and south can be seen an immense forest, stretching for miles around.  It is possible to climb down into the canyon from here.

spacy: ['you', 'top', 'canyon', 'wall', 'view', 'canyon', 'part', 'river', 'canyon', 'wall', 'cliffs', 'north', 'northwest', 'falls', 'rainbow', 'vision', 'i', 'top', 'dam', 'north', 'west', 'south', 'mile', 'it', 'canyon']
w2v: [('wall', -2.2868419), ('rainbow', -1.0936539), ('cliffs', -0.77175736), ('you', -0.68128884), ('west', -0.40599802), ('it', -0.3910706), ('south', -0.3569921), ('t

25 , This is the kitchen of the white house. A table seems to have been used recently for the preparation of food. A passage leads to the west, and a dark staircase can be seen leading upward. A dark chimney leads down and to the east is a small window which is open.

spacy: ['kitchen', 'house', 'table', 'preparation', 'food', 'passage', 'west', 'staircase', 'chimney', 'east', 'window']
w2v: [('chimney', -2.342733), ('window', -2.148847), ('table', -1.4394445), ('staircase', -1.2579949), ('kitchen', -0.5251174), ('house', -0.4864278), ('west', -0.40599802), ('east', -0.29441816), ('preparation', 0.14858748), ('food', 0.28863776), ('passage', 0.46332425)]
w2v u_c: [('passage', -0.73232067), ('kitchen', -0.6618335), ('chimney', -0.6110426), ('preparation', -0.5398145), ('window', -0.21067399), ('food', -0.13375717), ('table', -0.12136185), ('house', 0.11725928), ('west', 0.29344746), ('east', 0.35416582), ('staircase', 0.48625433)]
w2v brain_neuron [('chimney', -0.8754806), ('window', -0

30 , You are on the west edge of a chasm, the bottom of which cannot be seen. The east side is sheer rock, providing no exits. A narrow passage goes west. The path you are on continues to the north and south.
spacy: ['you', 'edge', 'chasm', 'bottom', 'side', 'rock', 'exit', 'passage', 'path', 'you', 'north', 'south']
w2v: [('bottom', -0.93304443), ('side', -0.7305247), ('you', -0.68128884), ('south', -0.3569921), ('chasm', -0.27506334), ('north', -0.22478718), ('path', -0.22209328), ('rock', -0.094619274), ('edge', 0.002740115), ('exit', 0.23883075), ('passage', 0.46332425)]
w2v u_c: [('you', -0.7581904), ('passage', -0.73232067), ('rock', -0.4618628), ('chasm', -0.39120466), ('path', -0.36832374), ('exit', -0.30657843), ('bottom', -0.13636088), ('edge', 0.009789526), ('south', 0.22169705), ('north', 0.31364602), ('side', 0.33414254)]
w2v brain_neuron [('path', -1.234421), ('bottom', -0.31777903), ('edge', -0.1560593), ('north', -0.12008931), ('south', -0.10186267), ('you', -0.08243513